In [1]:
# coding=utf-8
# -*- coding: cp936 -*-
import jieba
import jieba.posseg as pseg
import codecs
import re
import os
import time
import string
from nltk.probability import FreqDist
open=codecs.open
import jieba
import pandas as pd

In [2]:
from tool import *

In [3]:
test_data = pd.read_csv("test.csv", index_col=0, header=None)

In [4]:
test_data.columns=['content']
test_data.index.names=['row_id']

In [5]:
test_data.head()

,content
row_id,
1,比黑人碳头牙刷好用?
2,比之前坏的要好些，亮一些。
3,质量很好 用着很舒服 感觉有点小贵啊
4,发货速度很快，东西收到后基本上和描述的没什么差别，质量还行，比较实用，就是感觉有点渣脚，上面...
5,1、每星期至少两次掉线，一天重启几次也正常；2、5G已坏。


In [6]:
header=['row_id','content','theme','sentiment_word','sentiment_anls']

In [7]:
content_col="content"
word_split_result, word_class_result = split_word(test_data, content_col)

Building prefix dict from D:\Anaconda3\lib\site-packages\jieba\dict.txt ...
Loading model from cache C:\Users\gavin\AppData\Local\Temp\jieba.cache
Loading model cost 1.9808621406555176 seconds.
Prefix dict has been built succesfully.


In [8]:
test_data["word split"] = word_split_result
test_data["word class"] = word_class_result

In [9]:
word_class_result[:3]

['p/*/n/*/n/*/vn/*/a/*/p/*/x',
 'p/*/f/*/a/*/uj/*/v/*/m/*/x/*/v/*/m/*/x',
 'n/*/d/*/a/*/x/*/p/*/uz/*/d/*/a/*/x/*/n/*/n/*/a/*/a/*/y']

In [10]:
center_word_all_record, word_class_all_record=find_cW_eW(word_split_result, word_class_result)

In [11]:
test_data["theme"] = center_word_all_record
test_data["sentiment_word"] = word_class_all_record

In [12]:
anls_word_table = pd.read_csv("anls_word_table.csv", index_col=0, encoding="gbk")

In [13]:
anls_word_table.head()

,负面情感词语（中文）,正面情感词语（中文）,中性情感词语（中文）
满目疮痍,1.0,0.0,0.0
冷销,1.0,0.0,0.0
谒,0.0,1.0,0.0
烦,1.0,0.0,0.0
浪荡,1.0,0.0,0.0


In [14]:
sentiment_anls = find_emotion(word_class_all_record, anls_word_table)

['好']
['1']
['']
['']
['好', '小']
['1', '-1']
['光滑']
['1']
['坏']
['-1']


In [15]:
test_data["sentiment_anls"] = sentiment_anls

In [16]:
test_data.tail()

,content,word split,word class,theme,sentiment_word,sentiment_anls
row_id,,,,,,
19996,P4 ……但这本书的主要目的是让人有正确的婚姻观，让已婚者和未婚者按照圣经来认识婚姻的本质。...,P4/*/ /*/…/*/…/*/但/*/这/*/本书/*/的/*/主要/*/目的/*/是/...,eng/*/x/*/x/*/x/*/c/*/r/*/r/*/uj/*/b/*/n/*/v/*...,P4/*/婚姻观,正确/*/忠实,1/*/1
19997,说的是176长，想想可以够一人长，下单了。结果实测，只有170，只好委屈将来睡它的人了。将就吧。,说/*/的/*/是/*/176/*/长/*/，/*/想想/*/可以/*/够/*/一/*/人长...,v/*/uj/*/v/*/m/*/ns/*/x/*/v/*/c/*/v/*/m/*/n/*/...,,,
19998,一般，太软了，躺下去扁扁的，很低，给孩子用了,一般/*/，/*/太软/*/了/*/，/*/躺下去/*/扁扁的/*/，/*/很/*/低/*/...,a/*/x/*/a/*/ul/*/x/*/v/*/z/*/x/*/d/*/a/*/x/*/p...,,,
19999,很好用，大点就更好了,很好/*/用/*/，/*/大点/*/就/*/更好/*/了,d/*/p/*/x/*/n/*/d/*/d/*/ul,,,
20000,183 70穿L正好183 70穿L正好 喜欢宽松加大一码,183/*/ /*/70/*/穿/*/L/*/正好/*/183/*/ /*/70/*/穿/*...,m/*/x/*/m/*/v/*/eng/*/a/*/m/*/x/*/m/*/v/*/eng/...,L,正好,1


In [17]:
result = test_data[header[1:]]

In [18]:
result.columns

Index(['content', 'theme', 'sentiment_word', 'sentiment_anls'], dtype='object')

In [19]:
result.head()

,content,theme,sentiment_word,sentiment_anls
row_id,,,,
1,比黑人碳头牙刷好用?,黑人,好,1
2,比之前坏的要好些，亮一些。,,,
3,质量很好 用着很舒服 感觉有点小贵啊,质量/*/感觉,好/*/小,1/*/-1
4,发货速度很快，东西收到后基本上和描述的没什么差别，质量还行，比较实用，就是感觉有点渣脚，上面...,发货,光滑,1
5,1、每星期至少两次掉线，一天重启几次也正常；2、5G已坏。,G,坏,-1


In [20]:
def tranform(x):
    if len(x) > 0:
        return ";".join(x.split("/*/")) +";" 
    else:
        return x
        
result=result.iloc[:,[1,2,3]].applymap(tranform)

In [25]:
result.to_csv("result.csv",encoding="utf-8",index=True, header=True)

In [26]:
result.shape

(20000, 3)

In [27]:
result

,theme,sentiment_word,sentiment_anls
row_id,,,
1,黑人;,好;,1;
2,,,
3,质量;感觉;,好;小;,1;-1;
4,发货;,光滑;,1;
5,G;,坏;,-1;
6,中学;,好;,1;
7,产品;盒子;样子;,直接;久;差;,1;0;-1;
8,色;,旧;,-1;
9,老公;静音;,不错;充满;,1;1;
